In [27]:
import numpy as np
from preprocessing.text import tokenize
from database.connect import conn
from models.llm import load_llm
from models.classifier import load_classifier
from tqdm import tqdm
from preprocessing.image import preprocess_and_check_image


In [28]:

dataset_subset = tokenize("YES", tokenizer)


# Perform prediction
predictions = trainer.predict(dataset_subset)

predicted_class = np.argmax(predictions.predictions, axis=1)

print(predicted_class[0] <= 1)

100%|██████████| 1/1 [00:00<00:00, 417.88it/s]

True


In [29]:
llm = load_llm()
trainer, tokenizer = load_classifier()


In [30]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM Baby_Bath_Skin_and_Grooming") 
rows = cursor.fetchall()

In [31]:
IMAGE_INDEX = 4
PRIMARY_KEY = ['id']

columns = [col[0] for col in cursor.description]
pk_index = [i for i in range(len(columns)) if columns[i] in PRIMARY_KEY]

columns, pk_index

(['id',
  'name',
  'main_category',
  'sub_category',
  'image',
  'ratings',
  'no_of_ratings',
  'discount_price',
  'actual_price'],
 [0])

In [36]:
results_text = []
PK = []
indices = []

for i in tqdm(range(len(rows[:10]))):    
    image = preprocess_and_check_image(rows[i][IMAGE_INDEX])
    if image:
        prompt = "Analyze the picture, and the row data.\nDoes it have a baby picture on it that is seen easily in the package?"
        text_data = " | ".join([f"{columns[j]}: {rows[i][j]}" for j in range(len(columns)) if j != IMAGE_INDEX])
        result = llm.generate_content(
            [image, "\n\n", f"Row data: {text_data}\n\n", prompt]
        )
        results_text.append(result.text)
        dataset_subset = tokenize(result.text, tokenizer)
        predictions = trainer.predict(dataset_subset)

        predicted_class = np.argmax(predictions.predictions, axis=1)[0]
        
        if predicted_class <= 2:
            indices.append(i)
            PK.append([rows[i][j] for j in pk_index])
            

        

100%|██████████| 10/10 [00:33<00:00,  3.38s/it]


In [37]:
results_text, PK, indices

(["Yes, the LuvLap Liquid Cleanser bottle features a prominent picture of a baby playing with a stacking toy. It's clearly visible on the front label.\n",
  'No, there is no picture of a real baby on the packaging.  It has cartoon drawings of sea creatures and a stylized narwhal on the headrest.\n',
  "Yes, the packaging clearly displays a stylized image of a baby's head.  It's prominent near the top of the box and on the bottle label.",
  'No, there is no baby picture on the packaging. The image features a lotus flower and meadowsweet graphic.',
  "No, there isn't a clear picture of a baby on the Cetaphil Baby Mild Bar packaging.  There's a stylized image of a baby's bottom being washed with a cloth/puff and some bubbles, but not a distinct photo or drawing of a baby's face.\n",
  "No, there is no baby picture readily visible on the packaging in the image. The packaging shows the Johnson's logo and product information.",
  'No, there is no readily visible baby picture on the packaging

In [40]:
rows_filtered = [rows[i] for i in indices]

In [41]:
rows_filtered

[(11,
  'Luv Lap Liquid Cleanser, Anti-Bacterial, Food Grade, For Baby Bottles, Accessories and Vegetables, 1.5ltr',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/71p5fCtiCnL._AC_UL320_.jpg',
  Decimal('4.40'),
  '5,883',
  '₹509',
  '₹669'),
 (13,
  'The Moms Co. Natural Baby Massage Oil with 10 Oils - Sesame Oil, Avocado, Organic Almond, Organic Jojoba, Organic Chamomil...',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/71EQE76D+hL._AC_UL320_.jpg',
  Decimal('4.40'),
  '2,265',
  '₹331',
  '₹474'),
 (19,
  'BeeBaby Classic Powder Puff with Container for Baby (Blue) | Soft Touch. 100% BPA Free.',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/61+T2A8EfWL._AC_UL320_.jpg',
  Decimal('4.10'),
  '1,864',
  '₹121',
  '₹170'),
 (20,
  'Good knight 100% Natural Mosquito Repellent Fabric Roll-On | Paediatrician Certified | 8-Hour Protect | Cit

In [42]:
PK

[[11], [13], [19], [20]]